In [9]:
import requests
from bs4 import BeautifulSoup
import csv

# 크롤링할 첫 페이지 URL
base_url = 'https://news.daum.net/breakingnews/digital?page='

# 크롤링할 페이지 수 (원하는 만큼 설정 가능)
pages_to_crawl = 2

# 수집한 뉴스 URL들을 저장할 리스트
urls = []

# 페이지 반복
for page in range(1, pages_to_crawl + 1):
    # 각 페이지 URL 생성
    url = f'{base_url}{page}'
    
    # 웹페이지 요청 보내기
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 뉴스 리스트 엘리먼트 찾기
    news_list = soup.find_all('ul', class_='list_news2 list_allnews')
    
    # 뉴스 링크와 타이틀 추출
    for news_section in news_list:
        news_items = news_section.find_all('li')
        for item in news_items:
            link = item.find('a', class_='link_txt')['href']
            urls.append(link)  # URL 리스트에 추가

# 크롤링한 데이터를 저장할 리스트
data = []

# 각 URL에서 데이터를 크롤링
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 제목 크롤링
    title_tag = soup.find('h3', class_='tit_view')
    title = title_tag.get_text() if title_tag else ''
    
    # 작성자와 날짜 크롤링
    author_tag = soup.find('span', class_='txt_info')
    author = author_tag.get_text() if author_tag else ''
    date_tag = soup.find('span', class_='num_date')
    publication_date = date_tag.get_text() if date_tag else ''
    
    # 본문 크롤링 (p 태그에서 text만 추출, 특정 클래스의 p 태그 제외)
    paragraphs = soup.find_all('p')
    article_text = ''
    for paragraph in paragraphs:
        if 'class' not in paragraph.attrs:
            article_text += paragraph.get_text() + ' '
    
    # 출처 및 카테고리 크롤링
    publisher_id_tag = soup.find('a', {'data-tiara-action-name': 'GNB언론사명_클릭'})
    publisher_id = publisher_id_tag.get_text() if publisher_id_tag else ''

    category_id_tag = soup.find('ul', class_='gnb_comm')
    category_id = ''
    if category_id_tag:
        it_link = category_id_tag.find('a', {'data-tiara-layer': 'digital'})
        if it_link:
            category_id = it_link.get_text()

    # 데이터 저장
    data.append({
        'article_url': url,
        'title': title.strip(),
        'author': author.strip(),
        'publication_date': publication_date.strip(),
        'article_text': article_text.strip(),
        'publisher_id': publisher_id.strip(),
        'category_id': category_id.strip()
    })

# CSV 파일로 저장
csv_file = "news_data.csv"
csv_columns = ['article_url', 'title', 'author', 'publication_date', 'article_text', 'publisher_id', 'category_id']

try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for row in data:
            writer.writerow(row)
except IOError:
    print("I/O error")

print(f"크롤링 완료. '{csv_file}'에 데이터 저장됨.")


크롤링 완료. 'news_data.csv'에 데이터 저장됨.
